In [0]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'examples'))
	print(os.getcwd())
except:
	pass


  # TRF Tutorial

  In this notebook we show how to import word onset as a word-level feature and compute TRF from them.

  ## Import EEG

In [0]:
import os
from pyeeg.io import eeglab2mne
eeg_path = '/media/hw2512/SeagateExpansionDrive/EEG_data/Katerina_experiment/Processed/Fs-125/interp_bad/BP-0.3-65/Blink_pruned/'
eeg_fname = [f for f in os.listdir(os.path.join(eeg_path, list_subjects[3])) if f.endswith('.set')][0]
list_subjects = os.listdir(eeg_path)
raw = eeglab2mne(os.path.join(eeg_path, list_subjects[3], eeg_fname), load_ica=False)
raw = raw.filter(1, 15, n_jobs=2)


  ## Import Word-level features

In [0]:
# Import all paths
from pyeeg.io import WordLevelFeatures
env_path = '/media/hw2512/SeagateExpansionDrive/EEG_data/Katerina_experiment/story_parts/alignement_data/'
wordfreq_path = '/media/hw2512/SeagateExpansionDrive/EEG_data/Katerina_experiment/story_parts/word_frequencies/'
list_wordfreq_files = [item for item in os.listdir(wordfreq_path) if item.endswith('timed.csv')]
list_stories = [item.strip('_word_freq_timed.csv') for item in list_wordfreq_files]
list_env_files = [os.path.join(env_path, s, s + '_125Hz.Env') for s in list_stories]

# Loading word onset and duration for AUNP02:
wo_path = os.path.join(wordfreq_path, list_wordfreq_files[1])
duration_path = os.path.join(env_path, list_env_files[1])


In [0]:
# Create word-level feature object:
wf = WordLevelFeatures(path_praat_env=dur_path, path_wordonsets=wo_path)



  ## Run TRF modelling

In [0]:
from pyeeg.models import TRFEstimator


